### 카메라 모듈 화면 표시 & 얼굴인식

In [ ]:
from imutils import face_utils
import dlib
import cv2
import easydict

args = easydict.EasyDict({"shape_predictor": "./shape_predictor_68_face_landmarks.dat"})

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(args["shape_predictor"])

video = cv2.VideoCapture('http://192.168.0.190:81/stream')	#카메라모듈 스트리밍 주소
frame_size = (int(video.get(cv2.CAP_PROP_FRAME_WIDTH)), int(video.get(cv2.CAP_PROP_FRAME_HEIGHT)))

while True:
    ret, frame = video.read()
    if not ret:
        print("video road fail")
        break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    rects = detector(gray, 0)
    
    if (len(rects) > 0):
        text = "{} face(s) found".format(len(rects))
        cv2.putText(frame, text, (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        
    for rect in rects:
        (bX, bY, bW, bH) = face_utils.rect_to_bb(rect)
        cv2.rectangle(frame, (bX, bY), (bX + bW, bY + bH), (0, 255, 0), 1)
        
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
        
        for (i, (x, y)) in enumerate(shape):
            cv2.circle(frame, (x, y), 1, (0, 0, 255), -1)
            cv2.putText(frame, str(i + 1), (x - 10, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 0, 255), 1)
            
        cv2.imshow("Frame", frame)
        
        key = cv2.waitKey(1) & 0xFF
        
        if (key == ord("q")):
            break
    
    cv2.destroyAllwindows()
